In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [3]:
# Load data
raw_data = pd.read_excel('data/41467_2019_8325_MOESM3_ESM.xls', sheet_name=1)
raw_data.head()

,Catalyst Nr,Nr of publication,Cation 1,Cation 1 mol%,Cation 2,Cation 2 mol%,Cation 3,Cation 3 mol%,Cation 4,Cation 4 mol%,...,p(CH4)/p(O2),"p total, bar","Contact time, s","X(O2), %","X(CH4), %","S(COx), %","S(C2=), %","S(C2-), %","S(C2), %","Y(C2), %"
0,1,1,Mn,9.2,NaN,NaN,NaN,NaN,NaN,NaN,...,4.761905,1.013,0.04,NaN,11.0,NaN,NaN,NaN,45.500000,5.0050
1,2,1,Cd,4.7,NaN,NaN,NaN,NaN,NaN,NaN,...,4.761905,1.013,0.04,NaN,10.0,NaN,NaN,NaN,40.000000,4.0000
2,3,1,Sn,4.5,NaN,NaN,NaN,NaN,NaN,NaN,...,4.761905,1.013,0.04,NaN,8.4,NaN,NaN,NaN,4.800000,0.4032
3,4,1,Ti,10.4,NaN,NaN,NaN,NaN,NaN,NaN,...,4.761905,1.013,0.04,NaN,6.8,NaN,NaN,NaN,44.099998,2.9988
4,5,1,Pt,2.8,NaN,NaN,NaN,NaN,NaN,NaN,...,4.761905,1.013,0.04,NaN,8.0,NaN,NaN,NaN,1.300000,0.1040


In [11]:
# Pivoted tables
pivot_columns = ['Cation 1', 'Cation 2', 'Cation 3', 'Cation 4', 'Anion 1', 'Anion 2', 'Support 1', 'Support 2']
pivot_values = ['Cation 1 mol%', 'Cation 2 mol%', 'Cation 3 mol%', 'Cation 4 mol%', 
                'Anion 1 mol%', 'Anion 2 mol%', 'Support 1 mol%', 'Support 2 mol%']
pivot_tables = [raw_data.pivot(columns=column_i, values=value_i) for column_i, value_i in zip(pivot_columns, pivot_values)]
for list in pivot_tables:
    print(list.head())

Cation 1  Ag  Al  Au  Ba  Be  Bi  Ca   Cd  Ce  Co  ...  Tb  Th    Ti  Tm   V  \
0        NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN  ... NaN NaN   NaN NaN NaN   
1        NaN NaN NaN NaN NaN NaN NaN  4.7 NaN NaN  ... NaN NaN   NaN NaN NaN   
2        NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN  ... NaN NaN   NaN NaN NaN   
3        NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN  ... NaN NaN  10.4 NaN NaN   
4        NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN  ... NaN NaN   NaN NaN NaN   

Cation 1   W   Y  Yb  Zn  Zr  
0        NaN NaN NaN NaN NaN  
1        NaN NaN NaN NaN NaN  
2        NaN NaN NaN NaN NaN  
3        NaN NaN NaN NaN NaN  
4        NaN NaN NaN NaN NaN  

[5 rows x 56 columns]
Cation 2  NaN  Ag  Al   B  Ba  Be  Bi  Ca  Cd  Ce  ...  Tb  Th  Ti  Tm   V  \
0         NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN   
1         NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN   
2         NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN  

In [17]:
# Combine and calculate composition
concat_pivot_tables = pd.concat(pivot_tables, axis=1, sort=True)  # Concat all pivot tables into table with duplicate columns
composition = concat_pivot_tables.groupby(level=0, axis=1).sum()/100  # Groupby columns and add composition
composition.head()

,Ag,Al,Au,B,Ba,Be,Bi,Br,C,Ca,...,Te,Th,Ti,Tm,V,W,Y,Yb,Zn,Zr
0,0.0,0.908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.104,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Make clean dataframe
data_cleaned = pd.concat([
    raw_data.iloc[:, 1],  # Number of publication
    composition,  # Composition
    raw_data.iloc[:, 19:]  # Reaction data
], axis=1, sort=True)
data_cleaned.head()

,Nr of publication,Ag,Al,Au,B,Ba,Be,Bi,Br,C,...,p(CH4)/p(O2),"p total, bar","Contact time, s","X(O2), %","X(CH4), %","S(COx), %","S(C2=), %","S(C2-), %","S(C2), %","Y(C2), %"
0,1,0.0,0.908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.761905,1.013,0.04,NaN,11.0,NaN,NaN,NaN,45.500000,5.0050
1,1,0.0,0.953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.761905,1.013,0.04,NaN,10.0,NaN,NaN,NaN,40.000000,4.0000
2,1,0.0,0.955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.761905,1.013,0.04,NaN,8.4,NaN,NaN,NaN,4.800000,0.4032
3,1,0.0,0.896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.761905,1.013,0.04,NaN,6.8,NaN,NaN,NaN,44.099998,2.9988
4,1,0.0,0.972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.761905,1.013,0.04,NaN,8.0,NaN,NaN,NaN,1.300000,0.1040


In [19]:
# Save clean data
data_cleaned.to_excel('data/clean_data.xlsx')